In [16]:
import os
import json
import numpy as np
import cv2
from pathlib import Path

NB_DIR = Path().resolve()
ROOT = NB_DIR.parent
print(ROOT)

IMG_DIR = ROOT / "data/images2"
ANN_DIR = ROOT / "data/arrow_data/arrow_annotations"
OUT_IMG_DIR = ROOT / "data/arrow_data/images_out"
OUT_LABEL_DIR = ROOT / "data/arrow_data/labels_xy"

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

TARGET_W = 384
TARGET_H = 512

/Users/jleto/LocalProjects/rev_vision


In [8]:
for file in os.listdir(ANN_DIR):
    if not file.endswith(".json"):
        continue

    json_path = os.path.join(ANN_DIR, file)
    with open(json_path, "r") as f:
        data = json.load(f)

    img_filename = os.path.basename(data["imagePath"])
    img_path = os.path.join(IMG_DIR, img_filename)

    if not os.path.exists(img_path):
        print("Missing image:", img_path)
        continue

    img = cv2.imread(img_path)
    H, W = img.shape[:2]

    img_resized = cv2.resize(img, (TARGET_W, TARGET_H), interpolation=cv2.INTER_AREA)
    cv2.imwrite(os.path.join(OUT_IMG_DIR, img_filename), img_resized)

    sx = TARGET_W / W
    sy = TARGET_H / H

    coords = []

    for shape in data["shapes"]:
        x, y = shape["points"][0]
        coords.append([x * sx, y * sy])

    # coords is now [[lx,ly],[mx,my],[rx,ry]]
    coords = np.array(coords, dtype=np.float32).reshape(-1)

    out_name = file.replace(".json", ".npy")
    np.save(os.path.join(OUT_LABEL_DIR, out_name), coords)

    print("Saved:", img_filename, "and", out_name)

Saved: frame_0007.png and frame_0007.npy
Saved: frame_0011.png and frame_0011.npy
Saved: frame_0027.png and frame_0027.npy
Saved: frame_0026.png and frame_0026.npy
Saved: frame_0030.png and frame_0030.npy
Saved: frame_0010.png and frame_0010.npy
Saved: frame_0006.png and frame_0006.npy
Saved: frame_0021.png and frame_0021.npy
Saved: frame_0016.png and frame_0016.npy
Saved: frame_0020.png and frame_0020.npy
Saved: frame_0023.png and frame_0023.npy
Saved: frame_0019.png and frame_0019.npy
Saved: frame_0014.png and frame_0014.npy
Saved: frame_0018.png and frame_0018.npy
Saved: frame_0022.png and frame_0022.npy
Saved: frame_0029.png and frame_0029.npy
Saved: frame_0013.png and frame_0013.npy
Saved: frame_0005.png and frame_0005.npy
Saved: frame_0025.png and frame_0025.npy
Saved: frame_0009.png and frame_0009.npy
Saved: frame_0008.png and frame_0008.npy
Saved: frame_0032.png and frame_0032.npy
Saved: frame_0024.png and frame_0024.npy
Saved: frame_0004.png and frame_0004.npy
Saved: frame_002

In [19]:
import sys
import os

ROOT = os.path.abspath("..")    # one level up from the notebook
sys.path.append(ROOT)
from models.arrow_regression.arrow_dataset import ArrowXYDataset

ds = ArrowXYDataset(OUT_IMG_DIR, OUT_LABEL_DIR)

print("Dataset size:", len(ds))

img, coords = ds[0]
print("Image shape:", img.shape)      # expect (3,512,384)
print("Coords:", coords)              # expect 6 numbers

Dataset size: 25
Image shape: torch.Size([3, 512, 384])
Coords: tensor([135.7173, 305.6073, 162.8115, 291.7330, 191.2147, 295.2670])
